In [1]:
import tensorflow as tf
print(tf.__version__)


2022-07-04 15:12:07.386969: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-04 15:12:07.387037: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


2.7.0


In [2]:
import requests
import pandas as pd
import numpy as np


In [3]:
urlRedes = "http://vps-40d69db1.vps.ovh.ca:23004/api/TreinoRedes/obtemListaParaPrevisao"
urlExemplos = "http://vps-40d69db1.vps.ovh.ca:23004/api/ExemploTreinoAcaoEntradas/obtemProximoDia"
urlPrevisao = 'http://vps-40d69db1.vps.ovh.ca:23004/api/PrevisaoRedes/recebePrevisaoTreino'


In [4]:
def incluiEntrada(treino):
    urlEntrada = "http://vps-40d69db1.vps.ovh.ca:23004/api/CotacaoIntradayAcaoResultados/listaPrecoEntradaPrevisaoProximoB3?"\
        "idRegra=" + str(treino['regraProjecaoId']) + "&idGrupo=" + str(treino['grupoAcaoId'])
    print(urlEntrada)
    r = requests.get(urlEntrada)
    j = r.json()
    dfEntradas = pd.DataFrame.from_dict(j)
    return dfEntradas



In [5]:
def trataRede(rede):
    print('ID:' , rede['id'])
    nome_arquivo = '/root/executa-rede/redes/treino_rede' + str(rede['id']) + '.h5'
    print(nome_arquivo)
    modelo = tf.keras.models.load_model(nome_arquivo)
    modelo.summary()
    result = modelo.predict(X)
    dfDados['resultado'] = result
    dfDados['treinoRedeId'] = str(rede['id'])
    dfDados['tipoCompraVenda'] = rede['tipoCompraVenda']
    dfEntradas = incluiEntrada(rede)
    dfPrevisao = pd.merge(dfDados,dfEntradas, how='left', left_on='ticker' ,  right_on='ticker')
    json = dfPrevisao.to_dict(orient='list')
    print(json)
    #response = requests.post(urlPrevisao, data=json)
    #print(response)

In [6]:
def trataLinhaX(linha):
    return np.fromstring(linha['campoX'], dtype=float, sep=',')


In [7]:
print('conexao redes')
r = requests.get(urlRedes)
j = r.json()
dfRede = pd.DataFrame.from_dict(j)
dfRede.head()


conexao redes


,id,grupoAcaoId,periodoTreinoRedeId,tipoExemploTreinoId,regraProjecaoId,redeNeuralId,ativoPrevisao,tipoCompraVenda
0,2,12,1,1,30,2,1,V


In [8]:
print('conexao exemplos')
r = requests.get(urlExemplos)
j = r.json()
dfDados = pd.DataFrame.from_dict(j)
dfDados.head()


conexao exemplos


,campoX,ticker,diaNumPrevisao
0,"1.066763, 1.066763, 1.066763, 1.066763, 1.066...",ABEV3,20220704
1,"2.261905, 2.261905, 2.261905, 2.261905, 2.261...",AERI3,20220704
2,"1.661491, 1.661491, 1.661491, 1.661491, 1.661...",ALPA4,20220704
3,"1.128816, 1.128816, 1.128816, 1.128816, 1.128...",ALSO3,20220704
4,"1.809249, 1.809249, 1.809249, 1.809249, 1.809...",AMAR3,20220704


In [9]:
dfTratadoX = dfDados.apply(trataLinhaX, axis='columns')
X = np.stack(dfTratadoX.values)
X.shape


(88, 1800)

In [10]:
dfDados.drop('campoX', inplace=True, axis=1)
    

In [11]:
dfRede.apply(trataRede, axis='columns')


ID: 2
/root/executa-rede/redes/treino_rede2.h5


2022-07-04 15:12:40.274832: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-07-04 15:12:40.287514: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (vps34755): /proc/driver/nvidia/version does not exist
2022-07-04 15:12:40.384716: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-04 15:12:42.065946: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 12960000 exceeds 10% of free system memory.
2022-07-04 15:12:42.354227: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 12960000 exceeds 10% of free system memory.
2022-07-04 15:12:

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1800)              3241800   
                                                                 
 dense_1 (Dense)             (None, 1400)              2521400   
                                                                 
 dense_2 (Dense)             (None, 1000)              1401000   
                                                                 
 dense_3 (Dense)             (None, 600)               600600    
                                                                 
 dense_4 (Dense)             (None, 200)               120200    
                                                                 
 dense_5 (Dense)             (None, 1)                 201       
                                                                 
Total params: 7,885,201
Trainable params: 7,885,201
Non-

0    None
dtype: object